In [2]:
#읽은 데이터를 슬래쉬를 기준으로 태그와 함께 나열하는 데이터 전처리 과정 셀


from collections import defaultdict
import math

def sent_processing(lines):

    if isinstance(lines, list):
        lines = [line.strip().split(" ") for line in lines]

        corpus = []
        for line in lines:
            sent = []
            for word in line:
                word = tuple(word.rsplit("/", 1))
                sent.append(word)
            corpus.append(sent)

        return corpus

    elif isinstance(lines, str):
        line = []
        for word in lines.strip().split(" "):
            word = tuple(word.rsplit("/", 1))
            line.append(word)
        return line

    else:
        print("wrong type of input sentence")
        exit(1)

    
with open("corpus.txt", "r", encoding='utf-8') as f:
    lines = f.readlines()

corpus = sent_processing(lines) #처리된 데이터를 corpus에 저장

In [32]:
def train(corpus):

    def bigram_count(sent):
        poslist = [pos for _, pos in sent] # [NN, VBD, DT, NN]
        return [(pos0, pos1) for pos0, pos1 in zip(poslist, poslist[1:])]

    pos2words_freq = defaultdict(lambda: defaultdict(int)) # number of (word, tag)
    trans_freq = defaultdict(int) # bigram count --> (tag-1, tag)

    # sent format: [(word, tag), (word, tag), ...,(word, tag)]
    #corpus의 내부에서 반복문을 돈다.
    for sent in corpus: # counting
        for word, pos in sent: #튜플 하나하나 들어간다. 워드와 태그를 받음
            pos2words_freq[pos][word] +=1 
#책에서 참고 가능하듯이, {CMC:{아버지:10, 올림핌:15..}..} 처럼 딕셔너리 안에 딕셔너리(이중딕셔너리)로 한 태그에 대한 단어와 그 개수를 나열

        for bigram in bigram_count(sent):
            trans_freq[bigram] +=1
#책에서 참고 가능하듯이, {(CMC, fjb):20, ...}처럼 태그가 연속되는 것이 몇 개인지 체크하는 과정
# + 첫번쨰와 마지막 태그는 BOS와 EOS가 태그 중에 없으므로 따로 입력해서 추가해준다.

        trans_freq[('BOS', sent[0][1])] += 1 # number of (BOS, tag) bigram
        trans_freq[(sent[-1][1], 'EOS')] +=1 # number of (tag, EOS) bigram

    ### Practice1: emission prob p(x|y) 
    # base는 분모 (반복문으로 딕셔너리 안에 접근하여, 아버지를 구하고 싶으면, 아버지 개수 / CMC 개수 형태로 확률을 구한다.
    
    # base prob: p(y).
    base = {pos: sum(words_dic.values()) for pos, words_dic in pos2words_freq.items()}
    # P(y) for every y (count for each tag): {'CMC': count(CMC), 'CMP': count(CMP),..}  
    
    # p(x|y) = p(x, y) / p(y)
    pos2words_prob = defaultdict(lambda: defaultdict(float))
    for pos, words_dic in pos2words_freq.items():
        for word, count in words_dic.items():
            pos2words_prob[pos][word] = math.log(count/base[pos])
            
    # log(p(x, y)/p(y)) for every (x, y)
    # Do something..
    
    ### Practice2: transition prob p(y_t|y_(t-1))
    # base prob: p(y_(t-1))
    # (CMC,fgb)의 순서로 나온 개수 / CMC가 앞에 오는 경우의 전체합 으로 확률을 구한다.
    base = defaultdict(int)
    for (pos1, pos2), count in trans_freq.items():
        base[pos1] += count
    # Do something to make {'CMC': count('CMC'), 'fjb': count('fjb'), ..}

    # p(y_t|y_(t-1)) = p(y_t, y_(t-1)) / p(y_(t-1))
    trans_prob = {(pos1, pos2) : math.log(count/ base[pos1]) for (pos1,pos2), count in trans_freq.items()}
    # Do something -> p(y_t, y_t-1) / p(y_t) 
    
    return pos2words_prob, trans_prob

In [33]:
pos2words, trans = train(corpus)

print('명사 라면의 확률:', pos2words['CMC']['라면']) # 명사 '라면'의 확률 (신라면, 진라면 등.)
print('연결어미 라면의 확률:', pos2words['fmoc']['라면']) # 연결어미 '라면'의 확률 (~ 이라면)

명사 라면의 확률: -9.427948631791715
연결어미 라면의 확률: -5.6937321388027


In [38]:
class HMM_tagger(object):
    def __init__(self, pos2words, trans):
        self.pos2words = pos2words
        self.trans = trans
        self.unk = -15 #unknown word (모르는 단어) 학습했던 패턴이나 데이터에 등장하는 것 외에 등장하는 것
        self.eos ='EOS'
        self.bos ='BOS'

    def sent_log_prob(self, sent):
        # emission prob. (단어에 대한 태그 확률)
        log_prob = sum(
            (self.pos2words.get(tag, {}).get(word, self.unk) for word, tag in sent)
            # do someting..
         ) # get emission prob. for each (w, t), otherwise unk value

        # transition prob.
        bigrams = [(t0, t1) for (_, t0), (_, t1) in zip(sent, sent[1:])] # every bigram in sentence
        #하나씩 엇나가면서 묶은 튜플을 리스트로 만든 것
        log_prob+= sum(
            (self.trans.get(b, self.unk) for b in bigrams)
            # do something..
        )
        
        #get함수는 get(a)하면 key가 a인 것에 대한 value를 반환한다. 이 때, get(a,b)하면 key가 a인 value를 반환하되, 그 값이 없으면 b를 반환
        # bos
        log_prob += self.trans.get((self.bos, sent[0][1]), self.unk) # get BOS prob for the first (w, t)

        # eos
        log_prob += self.trans.get((sent[-1][1], self.eos), self.unk) # get EOS prob for the last (w, t)
        
        # length norm.
        log_prob /= len(sent)

        return log_prob

In [39]:
tagger = HMM_tagger(pos2words, trans)
test_sent1= "감기/CMC 는/fjb 줄이/YBD 다/fmof ./g"
test_sent2= "감기/fmotg 는/fjb 줄/CMC 이다/fjj ./g"
print("%s: %f" % (test_sent1, tagger.sent_log_prob(sent_processing(test_sent1))))
print("%s: %f" % (test_sent2, tagger.sent_log_prob(sent_processing(test_sent2))))

감기/CMC 는/fjb 줄이/YBD 다/fmof ./g: -5.489636
감기/fmotg 는/fjb 줄/CMC 이다/fjj ./g: -14.037157
